In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install stanza
!pip uninstall googletrans

!pip install langdetect
!pip install -U deep-translator
!pip install googletrans==3.1.0a0
!pip install google_trans_new

from textblob import TextBlob
import pandas as pd
import stanza
from deep_translator import GoogleTranslator
from googletrans import Translator
import time
import re
import csv


Fase di pre-processing
Input:
- File con le annotazioni
- Cartella con gli articoli da analizzare

In [ ]:
translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig
print(translated)

from googletrans import Translator
def newTranslator(text):

  translator = Translator()
  ar = translator.translate(text).text
  return ar


In [ ]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences


def get_sentences(text, lang):
    """
    Crea una pipeline Stanza per una lingua specifica, segmenta il testo in frasi e restituisce la lista di frasi.
    """
    stanza.download(lang)

    nlp = stanza.Pipeline(lang, processors='tokenize', use_gpu=True, verbose=False)

    doc = nlp(text)

    sentences = [sentence.text for sentence in doc.sentences]

    return sentences



def GetFraseDaAnalizzare(start, end, article, usa_stanza, lang):
    # Segmenta il testo
    if usa_stanza:
        sentences = get_sentences(article, lang)
    else:
        sentences = split_into_sentences(article)

    current_position = 0
    for sentence in sentences:
        sentence_length = len(sentence)
        if current_position <= start < current_position + sentence_length:
            return (sentence.strip())
        current_position += sentence_length + 1

    print("Nessuna frase trovata per offset:", start, end)
    return ""


def GetFraseDaAnalizzareConContesto(start, end, article, usa_stanza, lang):
    if usa_stanza:
        sentences = get_sentences(article, lang)
    else:
        sentences = split_into_sentences(article)

    current_position = 0
    for i, sentence in enumerate(sentences):
        sentence_length = len(sentence)
        if current_position <= start < current_position + sentence_length:
            previous_sentence = sentences[i - 1].strip() if i > 0 else ""
            next_sentence = sentences[i + 1].strip() if i < len(sentences) - 1 else ""
            target_sentence = sentence.strip()

            combined_text = f"{previous_sentence} {target_sentence} {next_sentence}".strip()
            return combined_text
        current_position += sentence_length + 1
    print("Nessuna frase trovata per offset:", start, end)
    return ""



def GetLingua(titolo):
  lingua='en'
  titolo=titolo.lower()
  if 'bg' in titolo:
    lingua='bg'
  elif 'en' in titolo:
    lingua='en'
  elif 'hi' in titolo:
    lingua='hi'
  elif 'pt' in titolo:
    lingua='pt'
  else:
    lingua='ru'

  return lingua

In [ ]:
##PROCEDURA PER OTTENERE IL DATASET FINALE PER L'ADDESTRAMENTO
import csv
import re
lingua='EN'
path_annotations="subtask-1-annotations.txt"
rows=[]

def clean_text_advanced(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    text = re.sub(r'[^\w\s.,!?]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

with open(path_annotations, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')

        if len(parts) > 6:
            parts = parts[:5] + [' '.join(parts[5:])]
        rows.append(parts)

df_split = pd.DataFrame(rows, columns=["File", "Name", "Start", "End","Role","Descriptor"])
righe_df_finale=[]
print(df_split)
for index, row in df_split.iterrows():
      path=row["File"]

      if row["Name"] is None:
        continue
      nome=row["Name"].lower()
      start=row["Start"]
      end=row["End"]
      role=row["Role"].lower()
      descriptor=row["Descriptor"].lower()
      try:
        with open(f'articoli/{path}', "r", encoding="utf-8") as file:
          articolo = file.read()
      except:
        continue

      lingua=GetLingua(path)
      frase=GetFraseDaAnalizzareConContesto(int(start),int(end),articolo,True,lingua)
      frase=newTranslator(frase)
      nome=newTranslator(nome)
      if frase=="":
        continue

      print(f"{frase}\n{nome}\n\n----------------")
      #INSERISCO frase, nome, start, end, role, descriptor in un nuovo dataframe risultante
      righe_df_finale.append([frase, nome, start, end,role,descriptor])
      print(index)


df = pd.DataFrame(righe_df_finale, columns=["Frase", "Name", "Start", "End","Role","Descriptor"])
df.to_csv(f'train_dataset_finale_{lingua}.csv', index=False, quoting=csv.QUOTE_ALL)

print(len(df))